<a href="https://colab.research.google.com/github/chiyeon01/Hands_On_Machine_Learning/blob/main/7%EC%9E%A5.%20%EC%95%99%EC%83%81%EB%B8%94%20%ED%95%99%EC%8A%B5%EA%B3%BC%20%EB%9E%9C%EB%8D%A4%20%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8/1-2.%20%EB%B3%B4%ED%8C%85%EA%B3%BC%20%EB%B0%B0%EA%B9%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 앙상블 학습
대중의 지혜(wisdom of the crowd)라는 이론을 이용한 방법.<br/>
일련의 예측기로부터 예측을 수집하면 가장 좋은 모델 하나보다 더 좋은 예측을 얻을 수 있다.<br/>
일련의 예측기를 **앙상블(ensemble)**이라고 부르기 때문에 이를 **앙상블 학습**이라고 함.

## 1.투표 기반 분류기
* hard voting :다수결로 정해지는 분류기
* soft voting : 개별 분류기의 예측 평균을 내어 확률이 가장 높은 클래스를 예측

### 직접 투표(Hard Voting)

In [5]:
from sklearn.datasets import make_moons
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

X, y = make_moons(n_samples=500, noise=0.3, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

voting_clf = VotingClassifier(
    estimators = [
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svc', SVC(random_state=42))
    ]
)

'''
sklearn에서는 VotingClassifier를 통해 투표 기반 앙상블을 수행할 수 있음.
이때 voting 매개변수를 'soft'로 바꿔주면 soft voting을 활용함.
'''

In [6]:
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))])

In [9]:
# 각 모델별 성능
for name, clf in voting_clf.named_estimators_.items():
    print(clf.score(X_test, y_test))

0.864
0.896
0.896


In [12]:
# 확실히 앙상블 학습이 더 좋은 것을 볼 수 있음.
voting_clf.score(X_test, y_test)

0.912

### 간접 투표(Soft Voting)
보통 직접 투표보다 간접 투표가 성능이 잘 나옴.

In [16]:
soft_voting_clf = VotingClassifier(
    estimators = [
        ('lr', LogisticRegression(random_state=42)),
        ('rf', RandomForestClassifier(random_state=42)),
        ('svc', SVC(probability=True, random_state=42))
    ],
    voting='soft'
)

soft_voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(probability=True, random_state=42))],
                 voting='soft')

In [17]:
soft_voting_clf.score(X_test, y_test)

0.92

## 배깅과 페이스팅
* 배깅(Bagging) : 훈련 세트에서 중복을 허용하여 샘플링하는 방식<br/>
* 페이스팅(Pasting) : 훈련 세트에서 중복을 허용하지 않고 샘플링하는 방식


### 사이킷런의 배깅과 페이스팅

In [18]:
'''
즉, 배깅과 페이스팅은 같은 알고리즘을 사용하고 훈련 세트의 서브셋을 랜덤으로 구성하여 각 분류기를 다르게 학습시키는 것이다.
'''

'\n즉, 배깅과 페이스팅은 같은 알고리즘을 사용하고 훈련 세트의 서브셋을 랜덤으로 구성하여 각 분류기를 다르게 학습시키는 것이다.\n'

In [19]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# 아래는 배깅이고, 페이스팅을 사용하려면 bootstrap=False로 지정하면 됨.
# n_jobs=-1은 가용한 모든 CPU 코어를 사용함.
# BaggingClassifier는 기반이 되는 분류기가 결정 트리 분류기처럼 클래스 확률을 추정할 수 있으면 soft voting을 사용.(안되면 hard voting)
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500,
                            max_samples=100, n_jobs=-1, random_state=42)

bag_clf.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=100,
                  n_estimators=500, n_jobs=-1, random_state=42)

### OOB(out-of-bag) 평가
* OOB : 배깅이나 페이스팅에서 각각의 학습기에 사용되지 않는 데이터를 의미함.
<br/>

앙상블의 평가는 각 예측기의 OOB 평가를 평균하여 얻는다.

In [22]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500,
                            max_samples=100, n_jobs=-1, oob_score=True, random_state=42)

bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

0.9253333333333333

In [23]:
bag_clf.score(X_test, y_test)

0.904

In [24]:
from sklearn.metrics import accuracy_score

# 책에서 sklearn.metrcis를 사용해서 그냥 한번 해봄..ㅎ
pred = bag_clf.predict(X_test)
accuracy_score(pred, y_test)

0.904